In [12]:
import pymysql
import time

in_src_dbname = 'serviceordercenter'
tb_name = 'df_sqlinfo'
in_betchnum = 200

"""定义连接地址"""
try:
    db_src = pymysql.connect(host='',
                             database='',
                             user='',
                             password='',
                             charset='utf8')

except Exception as db_error:
    print ("{}:源库连接失败：错误原因：{}".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),db_error))
try:
    db_tar = pymysql.connect(host='',
                             #port=4000,
                             database='',
                             user='',
                             password='',
                             charset='utf8')
except Exception as db_error:
    print ("{}:目标库连接失败：错误原因：{}".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),db_error))

"""开启游标(防止数据迁移占用连接时间太久，每张表单独开一次)"""
cur_src_dic = db_src.cursor(cursor=pymysql.cursors.DictCursor)
cur_tar = db_tar.cursor()

"""获取表所有列"""
sql_column_table = """SELECT COLUMN_NAME AS src_column_name,COLUMN_TYPE AS src_column_type
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_SCHEMA = \'{}\'
          AND TABLE_NAME = \'{}\'
        ORDER BY ORDINAL_POSITION;""".format(in_src_dbname,tb_name)

cur_src_dic.execute(sql_column_table)
src_columns = cur_src_dic.fetchall()
src_columnsss = ''
for src_column in src_columns:
    src_columnss = src_column.get('src_column_name')
    src_columnsss += '`' + src_columnss + '`,'
src_column_concat = src_columnsss.strip(',')

i = 0
sql_values_srcdata = """SELECT {}
            FROM `{}`
            LIMIT {},{};""".format(src_column_concat,tb_name,i,in_betchnum)
i = i + in_betchnum
sql_values_tardata = """INSERT INTO `{}` ({}) VALUES """.format(tb_name,src_column_concat)

cur_src_dic.execute(sql_values_srcdata)
dic_values_srcdata = cur_src_dic.fetchall()
if dic_values_srcdata:
    row_strc = ''
    for row_data in dic_values_srcdata:
        row_strs = ''
        for columnnt in src_columns:
            cv = row_data.get(columnnt.get('src_column_name'))
            ct = columnnt.get('src_column_type')
            if ct.count('char') > 0:
                if cv is not None:
                    cv = cv.replace('\\','\\\\').replace('\'','\\\'').replace('\"','\\\"')
                row_str = '\'' + str(cv) + '\''
            elif ct.count('text') > 0:
                if cv is not None:
                    cv = cv.replace('\\','\\\\').replace('\'','\\\'').replace('\"','\\\"')
                row_str = '\'' + str(cv) + '\''
            elif ct.count('time') > 0 or ct.count('date') > 0:
                row_str = '\'' + str(cv) + '\''
            elif ct.count('tinyint(1)') > 0:
                row_str = str(int(cv))
            elif ct.count('bit') > 0:
                row_str = int.from_bytes(cv,'little')
            elif ct.count('json') > 0 and cv != '' and cv is not None:
                row_str = str('\'' + str(cv.replace('\\','\\\\').replace('\'','\\\'').replace('\"','\\\"')) + '\'')
            else:
                row_str = str(cv)
            row_strs = row_strs + str(row_str) + ','
        row_strc = row_strc + '(' + row_strs.strip(',') + '),'
    sql_values_tardata = sql_values_tardata + str(row_strc.replace('None', 'NULL').replace('\'NULL\'', 'NULL').strip(',')) + ";"
    print (sql_values_tardata)
"""关闭游标"""
cur_src_dic.close()
cur_tar.close()
db_src.close()
db_tar.close()

INSERT INTO `df_sqlinfo` (`Id`,`DataEntityId`,`FormId`,`SqlKey`,`Content`,`CreatedById`,`CreatedAt`,`UpdatedById`,`UpdatedAt`,`DeletedById`,`DeletedAt`,`Deleted`) VALUES ('SI0000000104',NULL,'FI0000000102','Insert','[{\"Sql\":\"Insert Into test_dynamiclocal ( Id,`TestDynamicCode`,`TestDynamicName`,`TestTime`) Values (@[Id],@[TestDynamicCode],@[TestDynamicName],@[TestTime] );\",\"DatabaseName\":\"Default\",\"DataEntityCode\":\"test_dynamiclocal\"},{\"Sql\":\"Insert Into tb_workcarinfo ( Id,`VinNumber`) Values (@[Id],@[VinNumber] );\",\"DatabaseName\":\"\",\"DataEntityCode\":\"tb_workcarinfo\"}]','DynamicFormTest','2021-06-28 09:45:32',NULL,NULL,NULL,NULL,0),('SI0000000105',NULL,'FI0000000102','Detail','[{\"Sql\":\" Select  `Id`,  `TestDynamicCode`,  `TestDynamicName`,  `TestTime`  From test_dynamiclocal Where Id In @[Id];\",\"DatabaseName\":\"Default\",\"DataEntityCode\":\"test_dynamiclocal\"},{\"Sql\":\" Select  `Id`,  `VinNumber`  From tb_workcarinfo Where Id In @[Id];\",\"DatabaseNam

In [24]:
a = ([{"TABLE_NAME":"service"}])
print(a)

[{'TABLE_NAME': 'service'}]


In [26]:
for b in a:
    print(b.get('TABLE_NAME'))

service


In [28]:
a = 'service'
b = ([{"TABLE_NAME":"{}".format(a)}])
print (b)

[{'TABLE_NAME': 'service'}]


In [30]:
import pymysql
import time

in_src_dbname = 'serviceordercenter'

"""定义连接地址"""
try:
    db_src = pymysql.connect(host='',
                             database='',
                             user='',
                             password='',
                             charset='utf8')

except Exception as db_error:
    print ("{}:源库连接失败：错误原因：{}".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),db_error))
try:
    db_tar = pymysql.connect(host='',
                             #port=4000,
                             database='',
                             user='',
                             password='',
                             charset='utf8')
except Exception as db_error:
    print ("{}:目标库连接失败：错误原因：{}".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),db_error))

"""函数/过程/视图的迁移"""
"""迁移函数"""
cur_src_dic = db_src.cursor(cursor=pymysql.cursors.DictCursor)
"""获取所有函数"""
sql_get_function = """SELECT ROUTINE_NAME
           FROM INFORMATION_SCHEMA.ROUTINES
           WHERE ROUTINE_SCHEMA=\'{}\'
           AND  ROUTINE_TYPE = \'FUNCTION\';""".format(in_src_dbname)

cur_src_dic.execute(sql_get_function)
src_fun = cur_src_dic.fetchall()

for fun_name_dic in src_fun:
    fun_name = fun_name_dic.get('ROUTINE_NAME')
    """函数创建语句"""
    sql_show_function = """SHOW CREATE FUNCTION {};""".format(fun_name)
    print(sql_show_function)

    cur_src_dic.execute(sql_show_function)
    fun_desc = cur_src_dic.fetchone().get('Create Function')
    print(fun_desc)

    sql_create_function = """DROP FUNCTION IF EXISTS {};\n{}""".format(fun_name,fun_desc)
    print(sql_create_function)
    try:
        cur_tar.execute(sql_create_function)
        db_tar.commit()
        print ("{}:函数{}创建成功...".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())),fun_name))
        logger.debug("函数{}创建成功...".format(fun_name))
    except Exception as db_error:
        db_tar.rollback()
        logger.debug("函数{}创建失败，原因：{}".format(fun_name,db_error))
cur_src_dic.close()

SHOW CREATE FUNCTION fn_dataversionnumber;
CREATE DEFINER=`user_service`@`%` FUNCTION `fn_dataversionnumber`(InDataEntityCode varchar(50),
        InDataBaseName varchar(60)) RETURNS varchar(7) CHARSET utf8
    SQL SECURITY INVOKER
BEGIN
    DECLARE InVersionNumber VARCHAR(255);
    DECLARE OutVersionNumber VARCHAR(255);

    SELECT a.VersionNumber 
           INTO InVersionNumber    
    FROM df_dataentitiesinfo a
    INNER JOIN (SELECT MAX(Id)AS Id,
                       VersionNumber
                FROM df_dataentitiesinfo
                WHERE Deleted = 0
                GROUP BY DataEntityCode, DatabaseName) b
    ON a.Id = b.Id
    WHERE a.DataEntityCode = InDataEntityCode
    AND a.DatabaseName = InDataBaseName;

    #拼接字符串
    SELECT concat('V',substring_index(substring_index(InVersionNumber,'.',1),'V',-1)+1,'.0') INTO OutVersionNumber;

    IF OutVersionNumber IS NULL OR ''
    THEN RETURN 'V1.0';
    ELSE RETURN OutVersionNumber;
    END IF ;
	
END
DROP FUNCTION IF EXISTS f

NameError: name 'logger' is not defined

In [34]:
print('This is a \033[1;31m test \033[0m!')

This is a  test !


In [32]:
print('This is a \033[1;32;43m test \033[0m!')

This is a  test !


In [33]:
print('\033[1;33;44mThis is a test !\033[0m')

This is a test !


In [2]:
a = 1
b = 0
if a in (1,2,3):
    b = b + 1
print (b)

1


In [24]:
import pymysql
db_src = pymysql.connect(host='',
                         database='',
                         user='',
                         password='',
                         charset='utf8')
cur_src_dic = db_src.cursor(cursor=pymysql.cursors.DictCursor)
in_pre_funname = input("请输入需要迁移函数名：")
aa = cur_src_dic.execute('SELECT 1 FROM INFORMATION_SCHEMA.ROUTINES WHERE ROUTINE_SCHEMA=\'{}\' AND  ROUTINE_TYPE = \'FUNCTION\' AND ROUTINE_NAME = \'{}\' LIMIT 1;'.format('serviceordercenter',in_pre_funname))
a = cur_src_dic.fetchone().get('ROUTINE_NAME')
print (a)
b = 0
b = 1 if a == 1 else 2
print (b)

None
2


In [31]:
a = input("请输入a值：") or '1'
print(isinstance(a,int))
print(type(a))
b = 0
if a in ('1','2','3'):
    b = 1
else:
    b = 2
print (b)

False
<class 'str'>
2


In [ ]:
import pymysql

try:
    tidb_conn = pymysql.connect(host='gateway01.us-east-1.prod.aws.tidbcloud.com', port=4000, user='s49scd8yqRkHgCX.root', password='535nVmA4KIVVMTAT', database='test', ssl={'ca': '/etc/ssl/cert.pem'} )
except Exception as e:
    print('tidb连接错误原因：',e)